## Generate blogposts for all categories

# Blog-style Text Generator using Pre-trained Language Model
This notebook generates blog-like summaries that help users choose among products.
It takes clustered product data and creates human-readable reviews using a language model.

In [ ]:
def generate_blogposts_for_all_categories(
    df_combined,
    generator,
    n_shot_prompt=None,
    n_top_products=3,
    n_complaints=3,
    max_new_tokens=800,
    temperature=0.4,
    top_p=0.9,
    repetition_penalty=1.1
):
    """
    For each unique category in df_combined['clustered_category'], generates a blogpost using the provided generator pipeline.
    Prints the blogpost for each category.
    """
    from nltk.corpus import stopwords
    from collections import Counter

### Step 1: Import Required Libraries
These include utilities for data manipulation and the text generation model.

In [ ]:
# Helper to extract top complaints
    def get_top_complaints(df, product_name, n=n_complaints):
        neg_reviews = df[(df['name'] == product_name) & (df['sentiment'] == 'negative')]['combined_reviews']
        words = ' '.join(neg_reviews).split()
        stop_words = set(stopwords.words('english'))
        filtered_words = [w.lower() for w in words if w.lower() not in stop_words and len(w) > 2]
        most_common = Counter(filtered_words).most_common(n)
        return [word for word, _ in most_common]

### Step 2: Define Prompt Template (Few-shot)
This template guides the language model on how to structure the generated article.

In [ ]:
# Default n-shot prompt if not provided
    if n_shot_prompt is None:
        n_shot_prompt = """
--- Blogpost for E-readers ---
(Example blog post here...)

### Step 3: Define Text Generation Function
The `generate_blogposts_for_all_categories` function loops through product categories and generates a paragraph
of blog-style content based on the top-rated products, most common complaints, and overall quality.

In [ ]:
--- Blogpost for Smart Tablets ---
(Example blog post here...)

### Step 4: Loop Over Each Category
The model generates one blog post per category, combining the key information extracted from the reviews.

In [ ]:
--- Blogpost for Smart Home Essentials ---
(Example blog post here...)

### Step 5: Save or Print the Blog Posts
The final output can be used directly for customer-facing product recommendation platforms.

In [ ]:
--- Blogpost for Batteries ---
(Example blog post here...)
        """

In [ ]:
for category in df_combined['clustered_category'].unique():
        cat_df = df_combined[df_combined['clustered_category'] == category]
        if len(cat_df) < n_top_products:
            continue  # Skip small categories

In [ ]:
# Compute product scores
        product_scores = (
            cat_df.groupby('name')['sentiment_points']
            .agg(['mean', 'count'])
            .sort_values(by=['mean', 'count'], ascending=[False, False])
            .reset_index()
        )

In [ ]:
top_products = product_scores.head(n_top_products)
        worst_product = product_scores.tail(1)

In [ ]:
# Collect top complaints for top products
        complaints = {
            row['name']: get_top_complaints(cat_df, row['name'])
            for _, row in top_products.iterrows()
        }

In [ ]:
# Compose the specific prompt for the current category
        category_prompt = f"\n\n### Blogpost:\n\n"
        category_prompt += f"You are a product reviewer. Write a short, helpful blogpost for customers shopping for {category}.\n"
        category_prompt += "- The top 3 products are:\n"

In [ ]:
for i, row in top_products.iterrows():
            name = row['name']
            avg_rating = row['mean']
            review_count = int(row['count'])
            top_complaints = complaints[name]
            complaint_text = ', '.join(top_complaints) if top_complaints else 'Few complaints!'
            category_prompt += f"{i+1}. {name} (Avg. Rating: {avg_rating:.2f}, {review_count} reviews)\n"
            category_prompt += f"   Top complaints: {complaint_text}\n"

In [ ]:
# Include the worst product details
        worst_name = worst_product.iloc[0]['name']
        worst_rating = worst_product.iloc[0]['mean']
        category_prompt += f"\nThe worst product is {worst_name} (Avg. Rating: {worst_rating:.2f}).\n"
        category_prompt += "Explain why customers should avoid the worst product from the category, based on reviews.\n"
        category_prompt += "Write the blog entry in a friendly, informative tone.\n"

In [ ]:
# Final prompt
        prompt = n_shot_prompt + category_prompt

In [ ]:
# Generate blog post
        generated = generator(
            prompt,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=repetition_penalty
        )[0]['generated_text']

In [ ]:
# Extract model output after prompt
        blog_output = generated.split("Write the blog entry in a friendly, informative tone.")[-1].strip()

In [ ]:
print(f"\n--- Blogpost for {category} ---\n")
        print(blog_output)

In [ ]:
# Example usage
# clustered_data = pd.read_csv('path_to_your_clustered_data.csv')
# blog_content = generate_blog_post(clustered_data)
# save_blog_post(blog_content, 'outputs/blog_posts/recommendations_blog_post.md')